In [ ]:
import numpy as np
import pandas as pd
import math
import os
from utils import *
from sklearn.model_selection import train_test_split
from models import SVDplusplus


df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/sampleSubmission.csv")
args = read_config('config_models.yaml')


model = SVDplusplus(args)

#train, test = train_test_split(df, test_size=args.test_set_size, random_state=args.random_state)

model.train(df_train)

model.predict(df_test, output_file="output.csv")
